In [22]:
!pip install qiskit
!pip install qiskit_nature
!pip install qiskit_ibm_runtime
!pip install --prefer-binary pyscf
import numpy as np
import pickle as pkl
import os

# 02: H-Atom Fast Forward

Recent literature has shown methods such as Variational Fast Forwarding which aims to predict the long-term time evolutions of a system given data on the short-term evolutions of the system. Filip et al [1] and Cîrstoiu et al [2] have demonstrated two different ways to train a VFF using short-term evolution data. This dataset provides us short-term data and gives us testing points for long-term performance benchmarking

## 02.1 Data Format

The data generator gives us the following:
- `⎢ѱHK〉`: The Hartree Fock State of the system
- `X_train`: `[Δt, 2Δtm …MΔt]` which are uniform timestamps at which short-term evolutions are recorded
- `Y_train`: `U(Δt)⎢ѱHK〉...U(MΔt)⎢ѱHK〉` which are the short-term evolutions but with noise
- `X_test`: `[PΔt…NΔt]` which are the long-term timestamps
- `Y_test`: `U(PΔt)⎢ѱHK〉…U(NΔt)⎢ѱHK〉` which are the long-term noiseless evolutions

The data generator gets the `U(Δt)` and the `⎢ѱHK〉` from a H-atom model given by the hyperparameters. The noisy generations allow us to also benchmark our pipeline on how well it avoids overfitting to noise which wasn't discussed in the references above

### 02.1.1 H-Atom Hamiltonians

We need some Chemistry to first decide what geometries to import.

- H2: Usually found at a `0.735 Å` equilibrium bond distance [O’Malley et al. (2016)]
- H3+: Usually found both in an equilateral triangle configuration with `0.9 Å` between each pair
- H6: Usually modelled as a linear chain of 6 atoms with bond lengths `1 Å` between each pair


In [23]:
H2 = {"atom":"H 0 0 0; H 0 0 0.735", "basis":"sto-3g", "charge":0, "spin":0}
H3 = {"atom":"H 0 0 -0.45; H 0 0 0.45; H 0 0.78 0;", "basis":"sto-3g", "charge":1, "spin":0}
H6 = {"atom":"H 0 0 0; H 0 0 1; H 0 0 2; H 0 0 3; H 0 0 4; H 0 0 5;", "basis":"sto-3g", "charge":0, "spin":0}

#### Electronic Hamiltonians from PySCF for H2

In [24]:
from pyscf import gto
mol = gto.Mole()
mol.build(**H2)

Fermionic Hamiltonians are of the form:

$\hat{H}\;=\;E_{\text{nuc}}\;+\;\sum_{p q} h_{p q}\,a^{\dagger}_{p}\, a_{q}+\;\frac{1}{2}\,\sum_{p q r s}g_{p q r s}\,a^{\dagger}_{p}\, a^{\dagger}_{q}\,a_{r}\, a_{s}$

The first term is the constant nuclear repulsions term. The other terms are first quantization and second quantization terms for Fermionic Pair Creations and Anhilations. The co-efficients are derived from Molecular Orbital Integrals

##### Nuclear Repulsions Term

In [25]:
E_nuc = mol.energy_nuc()
E_nuc

np.float64(0.7199689944489797)

##### Solving Hartree Fock Iteratively

In [26]:
from pyscf import scf
mf = scf.RHF(mol).run()
mf

converged SCF energy = -1.116998996754


##### 1e and 2e Interactions in AO Basis

In [27]:
h_ao  = mf.get_hcore()
h_ao, h_ao.shape

(array([[-1.12421758, -0.9652574 ],
        [-0.9652574 , -1.12421758]]),
 (2, 2))

In [28]:
g_ao  = mol.intor('int2e')
g_ao, g_ao.shape

(array([[[[0.77460594, 0.44744572],
          [0.44744572, 0.57187698]],
 
         [[0.44744572, 0.3009177 ],
          [0.3009177 , 0.44744572]]],
 
 
        [[[0.44744572, 0.3009177 ],
          [0.3009177 , 0.44744572]],
 
         [[0.57187698, 0.44744572],
          [0.44744572, 0.77460594]]]]),
 (2, 2, 2, 2))

##### Converting to MO Basis

In [29]:
C      = mf.mo_coeff
h_mo   = C.T @ h_ao @ C
h_mo

array([[-1.25633907e+00, -1.37083854e-17],
       [-6.07732712e-17, -4.71896007e-01]])

In [30]:
from pyscf import ao2mo
g_mo_8fold = ao2mo.kernel(mol, C)
g_mo = ao2mo.restore(1, g_mo_8fold, C.shape[1])
g_mo

array([[[[ 6.75710155e-01,  1.09338783e-16],
         [ 1.09338783e-16,  6.64581730e-01]],

        [[ 1.39486891e-16,  1.80931200e-01],
         [ 1.80931200e-01, -1.03094037e-16]]],


       [[[ 1.39486891e-16,  1.80931200e-01],
         [ 1.80931200e-01, -1.03094037e-16]],

        [[ 6.64581730e-01,  2.57172666e-16],
         [ 2.57172666e-16,  6.98573723e-01]]]])

##### Converting to SO Basis

In [31]:
n_mo   = h_mo.shape[0]
n_so   = 2 * n_mo

h_so = np.zeros((n_so,n_so))
for p in range(n_mo):
    for q in range(n_mo):
        h_so[2*p  ,2*q  ] = h_mo[p,q]
        h_so[2*p+1,2*q+1] = h_mo[p,q]

g_so = np.zeros((n_so, n_so, n_so, n_so))
for p in range(n_mo):
    for q in range(n_mo):
        for r in range(n_mo):
            for s in range(n_mo):
                g_so[2*p  ,2*q  ,2*r  ,2*s  ] = g_mo[p,q,r,s]
                g_so[2*p+1,2*q+1,2*r+1,2*s+1] = g_mo[p,q,r,s]


In [32]:
os.makedirs("_Hamiltonians", exist_ok=True)
with open("_Hamiltonians/H2.bin", "wb") as f:
    pkl.dump((E_nuc, h_so, g_so), f)